# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine

import re
import numpy as np
import pandas as pd

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC

from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to /home/jens/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jens/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jens/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/jens/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response_database.db')
df = pd.read_sql('messages', engine)
df.head()
X = df['message']
Y = df.drop(labels=['genre', 'id', 'message', 'original'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)
    #replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english')) 
    clean_tokens = []
 
    
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    filtered_tokens = [w for w in clean_tokens if not w in stop_words] 
        
    return filtered_tokens

for i in range(10):
    print(tokenize(df['message'][i]))
    print(df['message'][i])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']
Weather update - a cold front from Cuba that could pass over Haiti
['hurricane']
Is the Hurricane over or is it not over
['looking', 'someone', 'name']
Looking for someone but no name
['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately']
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight']
says: west side of Haiti, rest of the country today and tonight
['information', 'national', 'palace']
Information about the National Palace-
['storm', 'sacred', 'heart', 'jesus']
Storm at sacred heart of jesus
['please', 'need', 'tent', 'water', 'silo', 'thank']
Please, we need tents and water. We are in Silo, Thank you!
['would', 'like', 'receive', 'message', 'thank']
I would like to receive the messages, thank you
['croix', 'de', 'bouqu

In [79]:
tokenize("that's a test")

['thats', 'test']

In [144]:
from sklearn.ensemble import AdaBoostClassifier

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [145]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)), 
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [146]:
print(pipeline)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f8d83c08620>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [147]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=42)


In [148]:
parameters = {
    #'clf__estimator__n_estimators': 10,
    #'clf__estimator__criterion': 'gini',
    #'clf__estimator__max_features': ('auto'),#, 'log2'),
    #'clf__estimator__min_samples_split': 2,
    'vect__ngram_range':  (1, 1),
    'tfidf__use_idf': False,
    'vect__max_df': 1.
}

pipeline.set_params(**parameters)

pipeline.fit(X_train, Y_train)
Y_pred = pipeline.predict(X_test)


In [67]:
#Y_pred = cv.best_estimator_.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [149]:
#print(cv.best_estimator_.score(X_test, Y_test))
print(pipeline.score(X_test, Y_test))

0.24121466768525593


In [85]:
Y_test_array = Y_test.to_numpy().T

In [98]:
Y_test.sum()

related                   3990
request                    945
offer                       25
aid_related               2208
medical_help               448
medical_products           276
search_and_rescue          150
security                    88
military                   169
water                      358
food                       581
shelter                    480
clothing                    74
money                      109
missing_people              56
refugees                   198
death                      238
other_aid                  697
infrastructure_related     355
transport                  242
buildings                  289
electricity                 96
tools                       29
hospitals                   59
shops                       27
aid_centers                 63
other_infrastructure       237
weather_related           1432
floods                     414
storm                      503
fire                        51
earthquake                 452
cold    

In [150]:
roc_score = []
for column in range(35):
    print(Y.columns[column])
    print(classification_report(Y_test_array[column], Y_pred.T[column]))
    roc = roc_auc_score(Y_test_array[column], Y_pred.T[column])
    print('roc auc score', roc)
    roc_score.append(roc)
    print(confusion_matrix(Y_test_array[column], Y_pred.T[column]))

related
              precision    recall  f1-score   support

       False       0.70      0.22      0.33      1246
        True       0.80      0.97      0.88      3990

    accuracy                           0.79      5236
   macro avg       0.75      0.59      0.60      5236
weighted avg       0.78      0.79      0.75      5236

roc auc score 0.5944876235532652
[[ 272  974]
 [ 117 3873]]
request
              precision    recall  f1-score   support

       False       0.89      0.97      0.93      4291
        True       0.76      0.48      0.59       945

    accuracy                           0.88      5236
   macro avg       0.83      0.73      0.76      5236
weighted avg       0.87      0.88      0.87      5236

roc auc score 0.7256652597598764
[[4148  143]
 [ 487  458]]
offer
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      5211
        True       0.33      0.04      0.07        25

    accuracy                           1

In [151]:
print('average roc', np.mean(roc_score)) #0.58

average roc 0.6430326626632457


### 6. Improve your model
Use grid search to find better parameters. 

In [158]:
from pprint import pprint

pprint(pipeline.get_params())

{'clf': MultiOutputClassifier(estimator=AdaBoostClassifier()),
 'clf__estimator': AdaBoostClassifier(),
 'clf__estimator__algorithm': 'SAMME.R',
 'clf__estimator__base_estimator': None,
 'clf__estimator__learning_rate': 1.0,
 'clf__estimator__n_estimators': 50,
 'clf__estimator__random_state': None,
 'clf__n_jobs': None,
 'memory': None,
 'steps': [('vect',
            CountVectorizer(tokenizer=<function tokenize at 0x7f8d83c08620>)),
           ('tfidf', TfidfTransformer(use_idf=False)),
           ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'tfidf': TfidfTransformer(use_idf=False),
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7f8d83c08620>),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': <class 'numpy.int64'>,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 

In [169]:

parameters = {
    'clf__estimator__n_estimators': [25, 50, 100],
    'clf__estimator__learning_rate': [1.],
    #'clf__estimator__max_features': ['auto'],#, 'log2'),
    #'clf__estimator__min_samples_split': [2],#, 3),#,
    'vect__ngram_range': [(1, 1)],#,(1,2)],#, (1, 2), (2,2)),
    'tfidf__use_idf': [True],#, False],
    #'vect__max_df': [1.],
    'vect__max_df': [1.0],
    #'clf__estimator__class_weight': ['balanced'],
}

grid_search = GridSearchCV(pipeline, param_grid=parameters, n_jobs=6, scoring='f1_macro', verbose=1)
grid_fit = grid_search.fit(X_train, Y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [170]:
grid_fit.best_params_

{'clf__estimator__learning_rate': 1.0,
 'clf__estimator__n_estimators': 100,
 'tfidf__use_idf': True,
 'vect__max_df': 1.0,
 'vect__ngram_range': (1, 1)}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [171]:
Y_pred = grid_fit.best_estimator_.predict(X_test)


In [172]:
roc_score = []
for column in range(35):
    print(Y.columns[column])
    print(classification_report(Y_test_array[column], Y_pred.T[column]))
    roc = roc_auc_score(Y_test_array[column], Y_pred.T[column])
    print('roc auc score', roc)
    roc_score.append(roc)
    print(confusion_matrix(Y_test_array[column], Y_pred.T[column]))

related
              precision    recall  f1-score   support

       False       0.68      0.29      0.40      1246
        True       0.81      0.96      0.88      3990

    accuracy                           0.80      5236
   macro avg       0.74      0.62      0.64      5236
weighted avg       0.78      0.80      0.77      5236

roc auc score 0.6219551688209287
[[ 357  889]
 [ 170 3820]]
request
              precision    recall  f1-score   support

       False       0.90      0.96      0.93      4291
        True       0.75      0.50      0.60       945

    accuracy                           0.88      5236
   macro avg       0.82      0.73      0.77      5236
weighted avg       0.87      0.88      0.87      5236

roc auc score 0.7338538025324323
[[4132  159]
 [ 468  477]]
offer
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      5211
        True       0.18      0.08      0.11        25

    accuracy                           0

[[4906   56]
 [ 228   46]]
direct_report
              precision    recall  f1-score   support

       False       0.87      0.96      0.91      4203
        True       0.71      0.44      0.54      1033

    accuracy                           0.85      5236
   macro avg       0.79      0.70      0.73      5236
weighted avg       0.84      0.85      0.84      5236

roc auc score 0.6964070977743965
[[4019  184]
 [ 582  451]]


In [173]:
print('average roc', np.mean(roc_score)) #0.647

average roc 0.6471730537166407


### 9. Export your model as a pickle file

In [96]:
import pickle
outfile = open('MLmodel.pkl', 'wb')
pickle.dump(grid_fit.best_estimator_, outfile)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.